In [1]:
from pybaseball import playerid_lookup
from pybaseball import statcast_single_game,statcast_pitcher,pitching_stats_bref

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import requests
from bs4 import BeautifulSoup as BS
import re
import warnings
import folium
import bokeh
from numpy import NaN
import seaborn as sns

from sklearn.linear_model import LinearRegression,ElasticNetCV,ARDRegression,Lasso, LogisticRegression
from sklearn.model_selection  import train_test_split,cross_val_score
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC,SVC,OneClassSVM

% matplotlib inline
pd.set_option('display.max_columns', 100)
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
playerid_lookup('kershaw', 'clayton')
kershaw_stats = statcast_pitcher('2019-04-01', '2019-05-01', 477132)
kershaw_stats.game_date = pd.to_datetime(kershaw_stats.game_date, errors='ignore',unit='s')
kershaw_stats.head()

Gathering player lookup table. This may take a moment.
Gathering Player Data


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,SL,2019-04-27,87.5,1.3929,6.2633,Clayton Kershaw,622569,477132,field_out,hit_into_play,NaN,NaN,NaN,NaN,5.0,"Pablo Reyes grounds out, third baseman Justin ...",R,R,L,LAD,PIT,X,5.0,ground_ball,3,2,2019,-0.5085,0.8145,-0.0987,2.2411,NaN,668804.0,NaN,2,7,Top,107.49,168.38,NaN,NaN,605131.0,NaN,190428_030236,-2.5475,-127.2540,-5.1874,-4.9424,25.7182,-22.6154,3.4405,1.6900,4.0,72.1,-27.0,87.001,2720.0,5.970,565813,477132,605131.0,571970.0,621035.0,457759.0,608369.0,592626.0,657077.0,641355.0,54.5299,0.047,0.055,0.0,1.0,0.0,0.0,2.0,47,6,Slider,0,1,1,0,1,0,1,0,Standard,Standard
1,SL,2019-04-27,87.8,1.3760,6.3322,Clayton Kershaw,622569,477132,NaN,foul,NaN,NaN,NaN,NaN,8.0,NaN,R,R,L,LAD,PIT,S,NaN,NaN,3,1,2019,-0.2224,0.9144,0.1878,2.1873,NaN,668804.0,NaN,2,7,Top,NaN,NaN,NaN,NaN,605131.0,NaN,190428_030201,-2.4057,-127.7938,-5.7503,-1.9196,25.6168,-21.3791,3.5012,1.6900,NaN,NaN,NaN,87.272,2454.0,5.880,565813,477132,605131.0,571970.0,621035.0,457759.0,608369.0,592626.0,657077.0,641355.0,54.6194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,5,Slider,0,1,1,0,1,0,1,0,Standard,Standard
2,SL,2019-04-27,87.5,1.3867,6.3464,Clayton Kershaw,622569,477132,NaN,ball,NaN,NaN,NaN,NaN,1.0,NaN,R,R,L,LAD,PIT,B,NaN,NaN,2,1,2019,-0.3639,0.7179,-0.7352,3.3197,NaN,668804.0,NaN,2,7,Top,NaN,NaN,NaN,NaN,605131.0,NaN,190428_030133,-4.3538,-127.4056,-2.6559,-3.1004,24.1893,-24.1395,3.4678,1.7173,NaN,NaN,NaN,87.263,2461.0,5.935,565813,477132,605131.0,571970.0,621035.0,457759.0,608369.0,592626.0,657077.0,641355.0,54.5643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,4,Slider,0,1,1,0,1,0,1,0,Standard,Standard
3,FF,2019-04-27,90.0,1.4448,6.3281,Clayton Kershaw,622569,477132,NaN,ball,NaN,NaN,NaN,NaN,12.0,NaN,R,R,L,LAD,PIT,B,NaN,NaN,1,1,2019,0.0354,1.6212,0.6634,3.6332,NaN,668804.0,NaN,2,7,Top,NaN,NaN,NaN,NaN,605131.0,NaN,190428_030111,-2.0088,-131.0202,-4.1199,0.8345,28.0367,-12.9955,3.4380,1.7772,NaN,NaN,NaN,89.776,2367.0,6.157,565813,477132,605131.0,571970.0,621035.0,457759.0,608369.0,592626.0,657077.0,641355.0,54.3427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,3,4-Seam Fastball,0,1,1,0,1,0,1,0,Standard,Standard
4,SL,2019-04-27,86.9,1.4425,6.3656,Clayton Kershaw,622569,477132,NaN,ball,NaN,NaN,NaN,NaN,13.0,NaN,R,R,L,LAD,PIT,B,NaN,NaN,0,1,2019,-0.5338,0.6767,-0.8995,1.2666,NaN,668804.0,NaN,2,7,Top,NaN,NaN,NaN,NaN,605131.0,NaN,190428_030047,-4.4954,-126.3385,-7.3276,-4.7284,25.1443,-23.8230,3.4632,1.8045,NaN,NaN,NaN,86.191,2538.0,5.833,565813,477132,605131.0,571970.0,621035.0,457759.0,608369.0,592626.0,657077.0,641355.0,54.6661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,2,Slider,0,1,1,0,1,0,1,0,Standard,Standard
